# Mouse management using OpenCV

This Jupyter Notebook executes a scritp that will allow you to move the cursor to the position your eyes are pointing at. By implementing an alternatve to mouse management that optimices daily work and modernizes the use of the conventional mouse, our productivity will be increased when developing.
#### Before you start!
- Notice that bad illumination will make a huge difference on the output due to the cascade model mistakes during the face detection process. 
- You should also have a good posture facing the camera since the model does not recognize a side-way face. Take a look at the expected results!

<img src="./images/demo.png" width="500">

### **<span style="color:green"><b><i>1. Libraries and pre-trained models imports.</i></b></span>**

The core library for this project is **OpenCV** since it will be responsable of processing each frame of the input  and make the corresponding transformations. Our program needs to be able to detect faces and eyes since the movement of the pupils are the responsable of the displacement of the mouse.
With that intention, by importing the open source **cascades** provided by OpenCV we will have two pre-trained models at our disposal.



<span style="color:red"><b><i>Attention! </i></b></span>You will have to install the libray **pynput** to run it locally on your compute. Otherwise, the mouse handler won't be able to detect your clicks. 

In [1]:
import numpy as np
import cv2
from pynput.mouse import Button, Controller

# We'll use the cascades from cv2 for face and eyes detecction
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Set the mouse handler
mouse = Controller()
# Var for pupil center
center = None
oldCenter = None

# Open a window with the input video
showResoults = 1 # Hide=0 and Show=1
# Sets the camera port
idxCamera = 0 # Default=0 

### **<span style="color:green"><b><i>2. Main iteration loop.</i></b></span>**

Since we have stablish the bases that our application will need to properly work, now we are able of **analyze** each frame of the input video. In this particular case this input is provided by our connected camera, so the program must follow  this sequence of steps:

-  First, the frame needs to be read from the camera and processed as grayscale for better detecction.
-  Then, we'll need to focus on the user's face through the previously imported face cascade. Within this new section the program looks for the eyes.
-  As before, the program will focus on the user's pupil within the eyes area. If more than two eyes are detected, only the most likely will be consider.
-  Applying the HoughCircles method we can detect the pupils with decent precission.
-  Calculing the difference between the center of consecutive frames we can determine how much the mouse pointer needs to be moved from its  position. 
- Last step is to check if the user has already press any click button. If so, use the controller provided by pynput to make the action.

<span style="color:red"><b><i>Attention! </i></b></span>Note that the program stops when the user presses the '**ESC**' key.

In [2]:
# Try to open the camera at idxCamera
capture = cv2.VideoCapture(idxCamera)
if not capture.isOpened(): 
    print('Camera could not be detected. Please try changing the port.')
    
while True:
    ret, frame = capture.read()

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect the face
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        # Draw the rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 5)
        roi_gray = gray[y:y+w, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
       
        # Detect the eyes
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)
        # Only detect two eyes per face
        if len(eyes) > 2: continue 
        for (ex, ey, ew, eh) in eyes:               
            # Draw the rectangle around the eyes
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 4)
            eyes_gray = roi_gray[ey:ey+ew, ex:ex+ew]
            eyes_color = roi_color[ey:ey+eh, ex:ex+ew]
            
            # Detect the pupil of the eyes
            circles = cv2.HoughCircles(eyes_gray, cv2.HOUGH_GRADIENT,1, ey/8, param1=400,param2=15,minRadius=int(ex/8),maxRadius=int(ex/3))
            if circles is not None:
                circles = np.uint16(np.around(circles))
                # only get one eye for mouse movement
                for i in circles[0]:
                    if len(circles) > 2: continue
                  # get the center of the circle
                    center = (i[0], i[1])
                  # draw the outer circle
                    cv2.circle(eyes_color,center,i[2],(0,255,0),1, 2)
                  # draw the center of the circle
                    cv2.circle(eyes_color,center,2,(0,0,255),3)
                    
                    # Calculate the mouse displacement
                    if oldCenter is not None:
                      displacement = (center[0] - oldCenter[0], center[1] - oldCenter[1])
                      if (displacement[0] < 10 and displacement[1] < 10):
                          mouse.move(displacement[0], displacement[1])
                    oldCenter = center
                    
    
    
    # Loop ends when pressing 'ESC'
    key = cv2.waitKey(20)
    if key == 27: 
        break
    if key == ord('q'):
      mouse.click(Button.left,1)
    if key == ord('e'):
      mouse.click(Button.right,1)
    
    # Controls the window display
    if (showResoults):
        cv2.imshow('Demo', frame)
        
capture.release()
cv2.destroyAllWindows()

### Conclusions and project improvements.
The hardest aspect is definitly **pupil traking**. Since our eyes are constantly moving and its displacement is very small, the HoughCircles() funtion is not precise enough. This ocasionates difficulties on the **mouse interactions** due to abrupt changes in the cursor position. An interesting way to improve this project could be change parameters values in search of an combination that correctly works. If it's not possible, maybe we should find another library.